*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd

train_data = pd.read_csv(train_file_path,delimiter='\t', header=None, index_col=False) 
valid_data = pd.read_csv(test_file_path,delimiter='\t', header=None, index_col=False)
valid_data


In [ ]:
#naming columns

train_data.columns =['label','message']
valid_data.columns =['label','message']
valid_data

In [ ]:
train_data.describe()

In [ ]:
#dropping duplicate values

train_data = train_data.drop_duplicates(subset ="message", keep = 'first')
valid_data = valid_data.drop_duplicates(subset ="message", keep = 'first')

In [ ]:
train_data.describe()

In [ ]:
# Get all the ham and spam messages
ham_train = train_data[train_data.label =='ham']
spam_train = train_data[train_data.label=='spam']

ham_test = valid_data[valid_data.label =='ham']
spam_test = valid_data[valid_data.label=='spam']

In [ ]:
from sklearn.utils import resample

#we have to handle imbalance data 
# Upsample minority class
spam_train_df = resample(spam_train, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(ham_train),    # to match majority class
                                 random_state=123) # reproducible results
ham_train_df= ham_train



spam_test_df = resample(spam_test, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(ham_test),    # to match majority class
                                 random_state=123) # reproducible results
ham_test_df= ham_test

print(ham_test_df.shape, spam_test_df.shape)

In [ ]:
#new df train
new_train_df = ham_train_df.append(spam_train_df).reset_index(drop=True)

# Get length column for each text
new_train_df['text_length'] = new_train_df['message'].apply(len)

#Calculate average length by label types
labels = new_train_df.groupby('label').mean()
labels

In [ ]:
#new df train
new_test_df = ham_test_df.append(spam_test_df).reset_index(drop=True)

# Get length column for each text
new_test_df['text_length'] = new_test_df['message'].apply(len)

#Calculate average length by label types
labels2 = new_test_df.groupby('label').mean()
labels2

In [ ]:
#get train_x, train_label
# Map ham label as 0 and spam as 1

new_train_df['msg_type']= new_train_df['label'].map({'ham': 0, 'spam': 1})
train_label = new_train_df['msg_type'].values
train_X=new_train_df['message'] 
train_label

In [ ]:
#test_x, test_label

new_test_df['msg_type']= new_test_df['label'].map({'ham': 0, 'spam': 1})
test_label = new_test_df['msg_type'].values
test_X= new_test_df['message'] 
test_label

In [ ]:
# Defining pre-processing hyperparameters
max_len = 60 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 800

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(train_X)

In [ ]:
# Get the word_index 
word_index = tokenizer.word_index
word_index

In [ ]:
# Sequencing and padding on training and testing 

training_padded = pad_sequences (tokenizer.texts_to_sequences(train_X), maxlen = max_len, padding = padding_type, truncating = trunc_type )
testing_padded = pad_sequences(tokenizer.texts_to_sequences(test_X), maxlen = max_len,padding = padding_type, truncating = trunc_type)

In [ ]:
# Shape of train tensor
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)


In [ ]:
print(training_padded[0])

In [ ]:
vocab_size = 800 
embeding_dim = 16
drop_value = 0.2 # dropout


In [ ]:
# Modeling 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional

#Dense model architecture
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(drop_value))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
# fitting a dense spam detector model
num_epochs = 15
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(training_padded, train_label, epochs=num_epochs, validation_data=(testing_padded, test_label),callbacks =[early_stop], verbose=2)

In [ ]:
# Model performance on test data 
model.evaluate(testing_padded, test_label)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

from itertools import chain
def predict_message(pred_text):
  temp= []
  temp.append(pred_text)
  new_seq = tokenizer.texts_to_sequences(temp)
  padded = pad_sequences(new_seq, maxlen =max_len,
                      padding = padding_type,
                      truncating=trunc_type)
  pred = model.predict(padded)


  pred = list(chain.from_iterable(pred))
  # prediction = []
  # prediction.append(pred[0])
  if pred[0] < .4 :
    pred.append('ham')
  elif pred[0] >= .4:
    pred.append('spam')

  
  return (pred)

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
prediction

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
